In [ ]:
!pip install tensorflow pandas numpy opencv-python matplotlib scikit-learn
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
image_folder = '/content/drive/MyDrive/dataset/train'

In [ ]:
import os
import numpy as np
import cv2

IMAGE_SIZE = (224, 224)

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load grayscale
    image = cv2.resize(image, IMAGE_SIZE)  # Resize for CNN
    image = np.stack((image,)*3, axis=-1)  # Convert grayscale to 3-channel
    image = image / 255.0  # Normalize pixel values
    return image

# Load all images from dataset
def load_images(image_paths):
    return np.array([preprocess_image(img) for img in image_paths], dtype=np.float32)


In [ ]:
# Function to process temperature CSV files
def preprocess_temperature(csv_path):
    try:
        df = pd.read_csv(csv_path, encoding="utf-16")  # Load CSV
        temp_features = df.select_dtypes(include=[np.number]).values.flatten()
        if temp_features.size > 0:
            return np.mean(temp_features), np.min(temp_features), np.max(temp_features)  # Extract key stats
    except:
        return None  # Return None if file is unreadable


In [ ]:
image_paths = []
temperature_features = []

for file in os.listdir(image_folder):
    base_name = os.path.splitext(file)[0]
    image_path = os.path.join(image_folder, file)
    csv_path = os.path.join(temp_folder, base_name + ".csv")

    if os.path.exists(csv_path):
        image_paths.append(image_path)
        temperature_features.append(preprocess_temperature(csv_path))

# Convert to NumPy arrays
temperature_features = np.array([t for t in temperature_features if t is not None], dtype=np.float32)

print(f"✅ Total Image Samples: {len(image_paths)}, Temperature Records: {len(temperature_features)}")


✅ Total Image Samples: 597, Temperature Records: 0


In [ ]:
sample_csv_path = os.path.join(temp_folder, os.listdir(temp_folder)[0])

# Read first 10 lines
with open(sample_csv_path, "r", encoding="utf-16") as file:
    sample_lines = [next(file) for _ in range(10)]

print("\n".join(sample_lines))


All temperatures in °C.



C:\Users\Siri\Documents\Fluke Connect\varunkotha20@gmail.com\01-20250227T085030Z-001\01\1.IS2



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246

In [ ]:
import pandas as pd
import numpy as np

def preprocess_temperature(csv_path):
    try:
        # Read CSV as text to check structure
        with open(csv_path, "r", encoding="utf-16") as file:
            lines = file.readlines()

        # Find the first numeric row (skip metadata)
        for idx, line in enumerate(lines):
            if line.strip() and line[0].isdigit():  # First line with numeric data
                data_start_idx = idx
                break

        # Read numerical data from the correct starting row
        df = pd.read_csv(csv_path, skiprows=data_start_idx, encoding="utf-16", header=None)

        # Extract temperature matrix as NumPy array
        temp_values = df.values.flatten()  # Convert to 1D array

        # Filter out any non-numeric values
        temp_values = temp_values[np.isfinite(temp_values)]

        # Return useful statistical features (mean, min, max)
        if temp_values.size > 0:
            return np.mean(temp_values), np.min(temp_values), np.max(temp_values)
        else:
            return None  # No valid temperature data found
    except Exception as e:
        print(f"⚠️ Error reading {csv_path}: {e}")
        return None


In [ ]:
image_paths = []
temperature_features = []

for file in os.listdir(image_folder):
    base_name = os.path.splitext(file)[0]
    image_path = os.path.join(image_folder, file)
    csv_path = os.path.join(temp_folder, base_name + ".csv")

    if os.path.exists(csv_path):
        temp_data = preprocess_temperature(csv_path)
        if temp_data is not None:
            image_paths.append(image_path)
            temperature_features.append(temp_data)

# Convert to NumPy array
temperature_features = np.array(temperature_features, dtype=np.float32)

print(f"✅ Total Image Samples: {len(image_paths)}, Temperature Records: {len(temperature_features)}")


In [ ]:
from sklearn.model_selection import train_test_split

train_paths, test_paths, train_temps, test_temps = train_test_split(
    image_paths, temperature_features, test_size=0.2, random_state=42
)

print(f"📌 Training Samples: {len(train_paths)}, Testing Samples: {len(test_paths)}")


📌 Training Samples: 477, Testing Samples: 120


In [ ]:
import numpy as np
import cv2

# Function to load and preprocess images
def load_images(image_paths):
    image_data = []
    for img_path in image_paths:
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        image = cv2.resize(image, (224, 224))  # Resize for CNN input
        image = image / 255.0  # Normalize pixel values
        image_data.append(image.reshape(224, 224, 1))  # Ensure shape consistency
    return np.array(image_data, dtype=np.float32)

# Load training and testing images
X_train_images = load_images(train_paths)
X_test_images = load_images(test_paths)

# Reshape temperature data for LSTM model
train_temps = train_temps.reshape(-1, 3, 1)  # (Samples, 3 features, 1 time step)
test_temps = test_temps.reshape(-1, 3, 1)

print(f"✅ Image Data Shape: {X_train_images.shape}, {X_test_images.shape}")
print(f"✅ Temperature Data Shape: {train_temps.shape}, {test_temps.shape}")


✅ Image Data Shape: (477, 224, 224, 1), (120, 224, 224, 1)
✅ Temperature Data Shape: (477, 3, 1), (120, 3, 1)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define CNN Model
def build_cnn_model():
    cnn_input = layers.Input(shape=(224, 224, 1), name="image_input")
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(cnn_input)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dense(1, activation="sigmoid")(x)  # Binary classification
    return keras.Model(cnn_input, x, name="CNN_Model")

cnn_model = build_cnn_model()
cnn_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
cnn_model.summary()


Model: "CNN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)             │ (None, 224, 224, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       6,422,592 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,517,377 (24.86 MB)

 Trainable params: 6,517,377 (24.86 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define LSTM Model for Temperature Data
def build_lstm_model():
    lstm_input = layers.Input(shape=(3, 1), name="temp_input")  # (3 features, 1 time step)
    y = layers.LSTM(32, return_sequences=True)(lstm_input)
    y = layers.LSTM(16)(y)
    y = layers.Dense(8, activation="relu")(y)
    return keras.Model(lstm_input, y, name="LSTM_Model")

# Define Hybrid CNN+LSTM Model
def build_hybrid_model():
    cnn_model = build_cnn_model()
    lstm_model = build_lstm_model()

    combined = layers.concatenate([cnn_model.output, lstm_model.output])
    z = layers.Dense(64, activation="relu")(combined)
    z = layers.Dense(32, activation="relu")(z)
    z = layers.Dense(1, activation="sigmoid")(z)  # Binary classification

    model = keras.Model(inputs=[cnn_model.input, lstm_model.input], outputs=z, name="Hybrid_Model")
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    return model

hybrid_model = build_hybrid_model()
hybrid_model.summary()


Model: "Hybrid_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)  │ (None, 224, 224, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 224, 224, 32)   │            320 │ image_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 112, 112, 32)   │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 112, 112, 64)   │         18,496 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 56, 56, 64)     │              0 │ conv2d_4[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 56, 56, 128)    │         73,856 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 28, 28, 128)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 100352)         │              0 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ temp_input (InputLayer)   │ (None, 3, 1)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 64)             │      6,422,592 │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 3, 32)          │          4,352 │ temp_input[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 32)             │          2,080 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 16)             │          3,136 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 1)              │             33 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 8)              │            136 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 9)              │              0 │ dense_5[0][0],         │
│                           │                        │                │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 64)             │            640 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)      

 Total params: 6,527,754 (24.90 MB)

 Trainable params: 6,527,754 (24.90 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Generate dummy labels (0 = healthy, 1 = diseased) for training
y_train = np.random.randint(0, 2, len(train_paths))
y_test = np.random.randint(0, 2, len(test_paths))

# Train Hybrid Model
history = hybrid_model.fit(
    [X_train_images, train_temps],  # Image + Temp input
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=16
)


Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.4657 - loss: 0.6991 - val_accuracy: 0.5104 - val_loss: 0.6930
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5225 - loss: 0.6918 - val_accuracy: 0.5104 - val_loss: 0.6946
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5096 - loss: 0.6941 - val_accuracy: 0.5104 - val_loss: 0.6947
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.5347 - loss: 0.6904 - val_accuracy: 0.5104 - val_loss: 0.6967
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.4867 - loss: 0.6960 - val_accuracy: 0.5104 - val_loss: 0.6968
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - accuracy: 0.5369 - loss: 0.6925 - val_accuracy: 0.5104 - val_loss: 0.6960
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.5263 - loss: 0.6910 - val_accuracy: 0.5104 - val_loss: 0.6978
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.5384 - loss: 0.6897 - val_accuracy: 0.5104 - v

In [ ]:
test_loss, test_acc = hybrid_model.evaluate(
    [X_test_images, test_temps],
    y_test
)
print(f"📌 Test Accuracy: {test_acc:.4f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.4856 - loss: 0.6957
📌 Test Accuracy: 0.5083
